In [1]:
"""Simulation of infections for different scenarios."""
import os
import covid19sim.coronalib as cl
import pandas as pd 
import numpy as np
import plotly.express as px
import datetime
import plotly.graph_objects as go
from plotly.offline import plot
from plotly.subplots import make_subplots

Covid-19 Sim provides two populations:<br>
"current" : The population is based on the current population (2019) <br>
"household" : The population is bases on a subsample in 2010 but with household numbers and additional persons per household

In [2]:
age, agegroup, gender, contacts, drate, hnr, persons = cl.makepop("household",17900000)

In [3]:
# Einlesen der realisierten Daten
nrw = pd.read_excel("./nrw_dat.xlsx")
nrw["Datum"] = nrw["Datum"].dt.date
nrw["Meldedatum"] = nrw["Meldedatum"].dt.date

# Szenarien mit Community Attack

## Basis Szenario NRW mit Community Attack

Die gesamte Repdouktionsetzt sich zusammen aus Infektionen innerhalb der Haushalte und Kontakt alle Haushaltsmitglieder mit der Aussenwelt. 
r_change ist hier jeweils der Kontaktwert mit der Aussenwelt

Die Kontakte sind proportional zur den Kontaktraten der Altersgruppen ohne Beschränkung.

In [ ]:
day0date = datetime.date(2020, 3, 8)
r_change = {}
r_change['2020-01-01'] = 3 * contacts/np.mean(contacts)
contacts_new = np.where(age < 20, contacts, contacts)
r_change['2020-03-08'] = 1.0 * contacts/np.mean(contacts)
r_change['2020-03-16'] = 0.3 * contacts/np.mean(contacts)
r_change['2020-03-23'] = 0.2 * contacts/np.mean(contacts)
com_attack_rate = {}
com_attack_rate["2020-01-1"] = 0.5
com_attack_rate["2020-05-4"] = 0.5

state, statesum, infections, day0, rnow, args, gr = cl.sim(
        age, drate, nday=180, prob_icu=0.009, day0cumrep=450,
        mean_days_to_icu=16, mean_duration_icu=14,
        mean_time_to_death=21,
        mean_serial=7.5, std_serial=3.0, immunt0=0.0, ifr=0.003,
        long_term_death=False, hnr=hnr, com_attack_rate=com_attack_rate,
        r_change=r_change, simname="NRW Basis",
        datadir=".",
        realized=nrw, rep_delay=13, alpha=0.125, day0date=day0date)

cl.plotoverview(gr, args)

In [ ]:
persgroup = np.where(persons>5,">5",persons)
cl.groupresults({"Personen":persgroup},state)

Der Anteil der Infizierten steigt deutlich mit der Haushaltsgröße. 
Dieses Bild sollte qualitativ recht realistisch sein. 

Es bestehen deutliche Hinweise, dass die Community Attack Rate 50% oder höher ist. 

In [ ]:
aux = cl.groupresults({"Alter":agegroup},state)
# Die CFR aus der IFR über die Dunkelziffer berechnen
aux["CFR"] = aux.IFR / args["alpha"]
display(aux)

In [ ]:
## Öffnung von Kitas

In [ ]:
day0date = datetime.date(2020, 3, 8)
r_change = {}
r_change['2020-01-01'] = 3 * contacts/np.mean(contacts)
contacts_new = np.where(age < 20, contacts, contacts)
r_change['2020-03-08'] = 1.0 * contacts/np.mean(contacts)
r_change['2020-03-16'] = 0.3 * contacts/np.mean(contacts)
r_change['2020-03-23'] = 0.2 * contacts/np.mean(contacts)
r_change['2020-04-20'] = 0.5 * contacts/np.mean(contacts)
r_change['2020-05-04'] = np.where(age<6,5*r_change['2020-04-20'],r_change['2020-04-20'])
com_attack_rate = {}
com_attack_rate["2020-01-1"] = 0.5
com_attack_rate["2020-05-4"] = 0.5

state, statesum, infections, day0, rnow, args, gr = cl.sim(
        age, drate, nday=180, prob_icu=0.009, day0cumrep=450,
        mean_days_to_icu=16, mean_duration_icu=14,
        mean_time_to_death=21,
        mean_serial=7.5, std_serial=3.0, immunt0=0.0, ifr=0.003,
        long_term_death=False, hnr=hnr, com_attack_rate=com_attack_rate,
        r_change=r_change, simname="NRW Basis",
        datadir=".",
        realized=nrw, rep_delay=13, alpha=0.125, day0date=day0date)

cl.plotoverview(gr, args)

Der Anteil der Infizierten ist hier in den jungen Altersgruppen höher. Für die Meldefälle könnte man eine Annahme treffen, dass die Wahrscheinlichkeit einer "Meldung" ebenfalls proportional zur statistischen Sterbewahrscheinlichkeit ist. 

Die CFR ist hier einfach als "Dunkeziffer" * IFR berechnet. Der Alterstrend sieht plausibel aus.

## Lockerung in allen Bereichen ausser bei den über 60 Jährigen

In [ ]:
day0date = datetime.date(2020, 3, 8)
r_change = {}
r_change['2020-01-01'] = 3 * contacts/np.mean(contacts)
contacts_new = np.where(age < 20, contacts, contacts)
r_change['2020-03-08'] = 1.0 * contacts/np.mean(contacts)
r_change['2020-03-16'] = 0.3 * contacts/np.mean(contacts)
r_change['2020-03-23'] = 0.2 * contacts/np.mean(contacts)
r_change['2020-05-04'] = 0.2 * np.where(age < 60, 3*contacts, contacts)/np.mean(contacts)

com_attack_rate = {}
com_attack_rate["2020-01-1"] = 0.5
com_attack_rate["2020-05-4"] = 0.5
state, statesum, infections, day0, rnow, args, gr = cl.sim(
        age, drate, nday=180, prob_icu=0.009, day0cumrep=450,
        mean_days_to_icu=16, mean_duration_icu=14,
        mean_time_to_death=21,
        mean_serial=7.5, std_serial=3.0, immunt0=0.0, ifr=0.003,
        long_term_death=False, hnr=hnr, com_attack_rate=com_attack_rate,
        r_change=r_change, simname="Lockerung U60",
        datadir=".",
        realized=nrw, rep_delay=13, alpha=0.125, day0date=day0date)

cl.plotoverview(gr, args)

Das effektive R steigt zwar leicht über 1 aber die Intesivbelastung ist weiter fallend, da die Maßnahmen für die ältere Bevölkerung weiter wirken. Es könnte jedoch sein, dass diese Annahme zu opimistisch ist.

In [ ]:
# Lockerung in allen Bereichen 

In [ ]:
day0date = datetime.date(2020, 3, 8)
r_change = {}
r_change['2020-01-01'] = 3 * contacts/np.mean(contacts)
contacts_new = np.where(age < 20, contacts, contacts)
r_change['2020-03-08'] = 1.0 * contacts/np.mean(contacts)
r_change['2020-03-16'] = 0.3 * contacts/np.mean(contacts)
r_change['2020-03-23'] = 0.2 * contacts/np.mean(contacts)
r_change['2020-05-04'] = 0.6 * contacts/np.mean(contacts)

com_attack_rate = {}
com_attack_rate["2020-01-1"] = 0.5
com_attack_rate["2020-05-4"] = 0.5
state, statesum, infections, day0, rnow, args, gr = cl.sim(
        age, drate, nday=180, prob_icu=0.009, day0cumrep=450,
        mean_days_to_icu=16, mean_duration_icu=14,
        mean_time_to_death=21,
        mean_serial=7.5, std_serial=3.0, immunt0=0.0, ifr=0.003,
        long_term_death=False, hnr=hnr, com_attack_rate=com_attack_rate,
        r_change=r_change, simname="Lockerung Alle",
        datadir=".",
        realized=nrw, rep_delay=13, alpha=0.125, day0date=day0date)

cl.plotoverview(gr, args)

Das effektive R ist ebenfalls leicht über 1, aber die Anzahl der ICU-Fälle steigt wieder stetig an. 

## Starke Lockerung aber Reduzierung der Community Attack Rate durch frühe Tests und Quarantäne

In [ ]:
day0date = datetime.date(2020, 3, 8)
r_change = {}
r_change['2020-01-01'] = 3 * contacts/np.mean(contacts)
contacts_new = np.where(age < 20, contacts, contacts)
r_change['2020-03-08'] = 1.0 * contacts/np.mean(contacts)
r_change['2020-03-16'] = 0.3 * contacts/np.mean(contacts)
r_change['2020-03-23'] = 0.2 * contacts/np.mean(contacts)
r_change['2020-05-04'] = 1.0 * contacts/np.mean(contacts)

com_attack_rate = {}
com_attack_rate["2020-01-1"] = 0.5
com_attack_rate["2020-05-4"] = 0.25
state, statesum, infections, day0, rnow, args, gr = cl.sim(
        age, drate, nday=180, prob_icu=0.009, day0cumrep=450,
        mean_days_to_icu=16, mean_duration_icu=14,
        mean_time_to_death=21,
        mean_serial=7.5, std_serial=3.0, immunt0=0.0, ifr=0.003,
        long_term_death=False, hnr=hnr, com_attack_rate=com_attack_rate,
        r_change=r_change, simname="Lockerung Alle kleinere CAR",
        datadir=".",
        realized=nrw, rep_delay=13, alpha=0.125, day0date=day0date)

cl.plotoverview(gr, args)

## Test der Community Attack Rate

In [ ]:
day0date = datetime.date(2020, 3, 8)
r_change = {}
r_change['2020-01-01'] = 3 * contacts/np.mean(contacts)
contacts_new = np.where(age < 20, contacts, contacts)
r_change['2020-03-08'] = 1.0 * contacts/np.mean(contacts)
r_change['2020-03-16'] = 0.3 * contacts/np.mean(contacts)
r_change['2020-03-23'] = 0.2 * contacts/np.mean(contacts)
r_change['2020-05-04'] = 0.2 * contacts/np.mean(contacts)

com_attack_rate = {}
com_attack_rate["2020-01-1"] = 0.5
com_attack_rate["2020-05-4"] = 0.1
state, statesum, infections, day0, rnow, args, gr = cl.sim(
        age, drate, nday=180, prob_icu=0.009, day0cumrep=450,
        mean_days_to_icu=16, mean_duration_icu=14,
        mean_time_to_death=21,
        mean_serial=7.5, std_serial=3.0, immunt0=0.0, ifr=0.003,
        long_term_death=False, hnr=hnr, com_attack_rate=com_attack_rate,
        r_change=r_change, simname="Test Community Attack",
        datadir=".",
        realized=nrw, rep_delay=13, alpha=0.125, day0date=day0date)

cl.plotoverview(gr, args)

# Szenarien ohne Community Attack
## Basisszenario NRW

In [ ]:
age, agegroup, gender, contacts, drate, hnr, persons = cl.makepop("current",17900000)

In [ ]:
day0date = datetime.date(2020, 3, 8)
r_change = {}
r_change['2020-01-01'] = 4.0 * contacts/np.mean(contacts)
r_change['2020-03-08'] = 0.24*8 * contacts/np.mean(contacts)
r_change['2020-03-16'] = 0.15*8 * contacts/np.mean(contacts)
r_change['2020-03-23'] = 0.10*6 * contacts/np.mean(contacts)
r_change['2020-05-04'] = 0.6 * contacts/np.mean(contacts)

com_attack_rate = {}
com_attack_rate["2020-01-1"] = 0.0
state, statesum, infections, day0, rnow, args, gr = cl.sim(
        age, drate, nday=180, prob_icu=0.009, day0cumrep=450,
        mean_days_to_icu=16, mean_duration_icu=14,
        mean_time_to_death=21,
        mean_serial=7.5, std_serial=3.0, immunt0=0.0, ifr=0.003,
        long_term_death=False, hnr=None, com_attack_rate=com_attack_rate,
        r_change=r_change, simname="Basis NRW ohne Community Attack",
        datadir=".",
        realized=nrw, rep_delay=13, alpha=0.125, day0date=day0date)

cl.plotoverview(gr, args)

In [ ]:
import pkg_resources
pkg_resources.get_distribution("covid19sim").version

In [ ]:
cl.plotoverview(gr, args)

In [ ]:
    gr["Wochentag"] = [x.weekday() for x in gr.Datum]
    gr["WE"] = np.where(gr.Wochentag > 4, "WE", "WT")
    fig = make_subplots(rows=2, cols=2)

    fig.add_trace(go.Scatter(x=gr["Datum"], y=gr["Erwartete Neu-Meldefälle"],
                             mode="lines", name="Erwartete Neu-Meldefälle"),
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=gr[gr.WE == "WE"]["Datum"],
                             y=gr[gr.WE == "WE"]["RKI Neu-Meldefälle"],
                             name="RKI Neu-Meldefälle (WE)",
                             mode="markers"), row=1, col=1)
    fig.add_trace(go.Scatter(x=gr[gr.WE == "WT"]["Datum"],
                             y=gr[gr.WE == "WT"]["RKI Neu-Meldefälle"],
                             name="RKI Neu-Meldefälle (WT)",
                             mode="markers"), row=1, col=1)

    fig.add_trace(go.Scatter(x=gr["Datum"], y=gr["Erwartete Gesamt-Meldefälle"],
                             name="Erwartete Gesamt-Meldefälle",
                             mode="lines"), row=2, col=1)
    fig.add_trace(go.Scatter(x=gr["Datum"], y=gr["RKI Gesamt-Meldefälle"],
                             name="RKI Gesamt-Meldefälle",
                             mode="lines"), row=2, col=1)

    fig.add_trace(go.Scatter(x=gr["Datum"], y=gr["Erwartete Tote"],
                             name="Erwartete Tote",
                             mode="lines"), row=1, col=2)
    fig.add_trace(go.Scatter(x=gr["Datum"], y=gr["IST Tote gesamt"],
                             name="Ist Tote gesamt",
                             mode="lines"), row=1, col=2)

    fig.add_trace(go.Scatter(x=gr["Datum"], y=gr["ICU"],
                             name="Erwartete Intensiv",
                             mode="lines"), row=2, col=2)
    fig.add_trace(go.Scatter(x=gr["Datum"], y=gr["Ist Intensiv"],
                             name="IST Intensiv",
                             mode="lines"), row=2, col=2)

    fig.update_layout(legend_orientation="h", title=args["simname"])

    plot(fig, filename=os.path.join(args["datadir"], args["simname"] +
                                    "_overview.html"),
         auto_open=False, auto_play=False)
    fig.show()

    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Scatter(x=gr["Datum"], y=gr["Reproduktionszahl"],
                             name="R effektiv",
                             mode="lines"), row=1, col=1)
    fig.add_trace(go.Scatter(x=gr["Datum"], y=gr["R extern"],
                             name="R extern",
                             mode="lines"), row=1, col=1)

    plot(fig, filename=os.path.join(args["datadir"], args["simname"] +
                                    "_reproduction.html"),
         auto_open=False, auto_play=False)
    plot(fig)

In [ ]:
day0date = datetime.date(2020, 3, 8)
r_change = {}
r_change['2020-01-01'] = 4.0 * contacts/np.mean(contacts)
r_change['2020-03-08'] = 0.24*8 * contacts/np.mean(contacts)
r_change['2020-03-16'] = 0.15*8 * contacts/np.mean(contacts)
r_change['2020-03-23'] = 0.9 * contacts/np.mean(contacts)
r_change['2020-05-04'] = 0.9 * contacts/np.mean(contacts)

com_attack_rate = {}
com_attack_rate["2020-01-1"] = 0.0
state, statesum, infections, day0, rnow, args, gr = cl.sim(
        age, drate, nday=180, prob_icu=0.009, day0cumrep=450,
        mean_days_to_icu=16, mean_duration_icu=14,
        mean_time_to_death=21,
        mean_serial=7.5, std_serial=3.0, immunt0=0.0, ifr=0.003,
        long_term_death=False, hnr=None, com_attack_rate=com_attack_rate,
        r_change=r_change, simname="Basis NRW ohne Community Attack",
        datadir=".",
        realized=nrw, rep_delay=13, alpha=0.125, day0date=day0date)

cl.plotoverview(gr, args)